![CMCC](http://cmcc.ufabc.edu.br/images/logo_site.jpg)

# **Lab 4c - Mineração de Textos**

#### Um outro tipo de base de dados que necessita de tratamento especial para representá-la de forma a ser utilizada em algoritmos de aprendizado são as bases textuais.

#### Neste notebook aprenderemos a gerar representações utilizadas na literatura.

#### As células-exercícios iniciam com o comentário `# EXERCICIO` e os códigos a serem completados estão marcados pelos comentários `<COMPLETAR>`.

#### ** Nesse notebook: **
#### *Parte 1:* Bag-of-Words
#### *Parte 2:* Bag-of-Words ponderados: TF-IDF
#### *Parte 3:* Similaridade do Cosseno
#### *Parte 4:* k-NN para documentos textuais

### **Parte 0: Preliminares**

#### Para este notebook utilizaremos a base de dados Movie Reviews que será utilizada para o segundo projeto.

#### A base de dados tem os campos separados por '\t' e o seguinte formato:
   `"id da frase","id da sentença","Frase","Sentimento"`

#### Para esse laboratório utilizaremos apenas o campo "Frase".

In [ ]:
import os
import numpy as np

def parseRDD(point):
    """ Parser for the current dataset. It receives a data point and return
        a sentence (third field).
    Args:
        point (str): input data point
    Returns:
        str: a string
    """    
    data = point.split('\t')
    return (int(data[0]),data[2])

def notempty(point):
    """ Returns whether the point string is not empty
    Args:
        point (str): input string
    Returns:
        bool: True if it is not empty
    """   
    return len(point[1])>0

filename = os.path.join("Data","Aula04","MovieReviews2.tsv")
rawRDD = sc.textFile(filename,2)
header = rawRDD.take(1)[0]

dataRDD = (rawRDD
           .filter(lambda x: x!=header)
           .map(parseRDD)
           .filter(notempty)
           #.sample( False, 0.1, 42 )
           )

print 'Read {} lines'.format(dataRDD.count())
print 'Sample line: {}'.format(dataRDD.takeSample(False, 1)[0])

### **Parte 1: Bags of Words**

#### Uma abordagem simples é transformar as sentenças em vetores esparsos de maneira similar ao One-Hot Encoding. Para isso utilizaremos o conceito de [Bag-of-words][bag-of-words].

####  A ideia geral é tratar cada **documento** como uma coleção desordenada de **tokens**.

#### Um token é um elemento extraído do texto. Inicialmente podemos pensar neles como uma única palavra, porém temos outros tipos de tokens possíveis conforme veremos mais para frente.

[bag-of-words]: https://en.wikipedia.org/wiki/Bag-of-words_model

### **1(a) Tokenizando a String**

#### Implemente a função `simpleTokenize(string)` que recebe uma string e retorna uma lista de tokens (palavras) não-vazias dessa string.  `simpleTokenize` deve utilizar expressões regulares para podermos generalizar a limpeza das strings nos próximos passos. Utilizando a função [re.split()](https://docs.python.org/2/library/re.html#re.split) retorne a lista de tokens não-vazios, convertidos em caixa baixa utilizando expressão regular a ser inserida na variável `split_regex`.

In [ ]:
# EXERCICIO
import re

quickbrownfox = 'A quick brown fox jumps over the lazy dog.'
split_regex = r'\W+'

def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    return <COMPLETAR>

print simpleTokenize(quickbrownfox) # Should give ['a', 'quick', 'brown', ... ]

In [ ]:
# TEST Tokenize a String (1a)
assert simpleTokenize(quickbrownfox)==['a','quick','brown','fox','jumps','over','the','lazy','dog'], 'lista incorreta!'
print 'ok!'
assert simpleTokenize(' ') == [], 'lista incorreta!'
print 'ok!'
assert simpleTokenize('!!!!123A/456_B/789C.123A')== ['123a','456_b','789c','123a'],'lista incorreta!'
print 'ok!'
assert simpleTokenize('fox fox') == ['fox', 'fox'],'lista incorreta'
print 'ok!'

### **(1b) Removendo stopwords**
#### *[Stopwords][stopwords]* são palavras comuns que não contribuem para o significado do texto (ex.: "e", "um", "uma", "é", etc.). Permitir que essas palavras permaneçam no texto introduz ruídos durante a comparação de bases textuais.

#### Usando o arquivo "stopwords.txt", implemente  `tokenize`, uma versão melhorada da função anterior que remove as stopswords.
[stopwords]: https://en.wikipedia.org/wiki/Stop_words

In [ ]:
# EXERCICIO
stopfile = os.path.join("Data","Aula04","stopwords.txt")
stopwords = set(sc.textFile(stopfile).collect())
print 'These are the stopwords: %s' % stopwords

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    return <COMPLETAR>

print tokenize(quickbrownfox) # Should give ['quick', 'brown', ... ]

In [ ]:
# TEST Removing stopwords (1b)
assert tokenize("Why a the?") == [], 'valores incorretos'
print 'ok!'
assert tokenize("Being at the_?") == ['the_'], 'valores incorretos'
print 'ok!'
assert tokenize(quickbrownfox) == ['quick','brown','fox','jumps','lazy','dog'], 'valores incorretos'
print 'ok!'

### **(1c) Tokenizando a base de dados**

#### Vamos aplicar nosso tokenizador em nossa base de dados e contar o número total de tokens e criar uma função que conta quantas tokens existem em nossa base.

In [ ]:
# EXERCICIO
dataToToken = dataRDD.<COMPLETAR>

def countTokens(textRDD):
    """ Count and return the number of tokens
    Args:
        textRDD (RDD of tokens): RDD containing tokens for 
    Returns:
        count: count of all tokens
    """
    return (textRDD
            .<COMPLETAR>
            .<COMPLETAR>
            ).collect()[0][1]

totalTokens = countTokens(dataToToken)
print 'There are %s tokens in the dataset' % totalTokens

In [ ]:
# TEST Tokenizing the small datasets (1c)
assert totalTokens == 80502,  'valor incorreto'
print 'ok!'

### **(1d) Registro com maior número de tokens**

#### Ordene o RDD e encontre o registro com maior número de tokens.

In [ ]:
# EXERCICIO
def findBiggestRecord(textRDD):
    """ Find and return the record with the largest number of tokens
    Args:
        textRDD (RDD of (recordId, tokens)): input Pair Tuple of record ID and tokens
    Returns:
        list: a list of 1 Pair Tuple of record ID and tokens
    """
    return (textRDD            
            .<COMPLETAR>
            ).take(1)

biggestRecord = findBiggestRecord(dataRDD)
print 'The record with ID "%s" has the most tokens (%s)' % (biggestRecord[0][0],
                                                                   len(biggestRecord[0][1]))

In [ ]:
# TEST Amazon record with the most tokens (1d)
assert 105156==biggestRecord[0][0], 'valor incorreto'
print 'ok'
assert 283==len(biggestRecord[0][1]), 'valor incorreto'
print 'ok'

### **Parte 2: Bag-of-Words Ponderados usando TF-IDF**

#### Particularmente em textos grandes nem todos os tokens tem a mesma importância na descrição de um texto. Ponderações nos ajudam a mensurar a importância de cada elemento de um vetor. Uma heurística para medir a importância dos tokens é a chamada "Term-Frequency/Inverse-Document-Frequency", ou [TF-IDF][tfidf].

#### **TF**
#### TF diz que a importância de um token é proporcional a frequência com que ele aparece no documento. Ou seja, se um documento *d* contém 100 tokens  e o token *t* aparece em *d* por 5 vezes, então o peso TF de *t* em *d* é *5/100 = 1/20*. A intuição é de que se um token ocorre com mais frequência do que os outros tokens, ele deve ter um contexto mais importante.

#### **IDF**
#### IDF pondera os tokens proporcional ao inverso da frequência dele na base de dados. Ou seja, se dois documentos possuem um termo em comum que é frequente, essa comparação não terá um peso tão grande quanto dois documentos que compartilham um termo raro. IDF de um token *t*, em um conjunto de documentos, *U*, é calculado como:

* #### Seja *N* a quantidade total de documentos em *U*
* #### Encontre *n(t)*, o número de documentos em *U* que contém *t*
* #### *IDF(t) = N/n(t)*.

#### **TF-IDF**
#### Finalmente, o peso TF-IDF é o produto entre TF e IDF de um termo.

[tfidf]: https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### **(2a) Implemente a função TF**

#### Implemente `tf(tokens)` que recebe uma lista de tokens e retorna um dicionário Python [dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries) mapeando os tokens para pesos TF.

#### Os passos que a função deve realizar:
* #### Crie um dicionário vazio
* #### Para cada token na lista `tokens`, conte 1 para cada ocorrência e some o token ao dicionário
* #### Para cada token no dicionário, divida o total de ocorrência do token pelo total de tokens

In [ ]:
# EXERCICIO
def tf(tokens):
    """ Compute TF
    Args:
        tokens (list of str): input list of tokens from tokenize
    Returns:
        dictionary: a dictionary of tokens to its TF values
    """
    freq = {}
    if len(tokens)==0:
        return {}
    inc = <COMPLETAR>
    for token in tokens:
        freq[token] = <COMPLETAR>
    return freq

print tf(tokenize(quickbrownfox)) # Should give { 'quick': 0.1666 ... }

In [ ]:
# TEST Implement a TF function (2a)
tf_test = tf(tokenize(quickbrownfox))
assert tf_test == {'brown': 0.16666666666666666, 'lazy': 0.16666666666666666,'jumps': 0.16666666666666666, 'fox': 0.16666666666666666,
                             'dog': 0.16666666666666666, 'quick': 0.16666666666666666},'valores incorretos'
print 'ok'
tf_test2 = tf(tokenize('one_ one_ two!'))
assert tf_test2 == {'one_': 0.6666666666666666, 'two': 0.3333333333333333},'valores incorretos'
print 'ok'

### **(2b) Implemente a função IDF**

#### Implemente `idfs` que recebe um RDD de textos como parâmetro e calcula o peso IDF de cada token único. A função deve retornar uma RDD de tupla onde a chave é o token e o valor seu IDF correspondente.

#### Lembre-se que o peso IDF de um token *t*, em um conjunto de documentos, *U*, é calculado da seguinte forma:
* #### Calcule *N* como o total de registros em *U*.
* #### Encontre *n(t)*, o número de documentos em *U* que contém *t*.
* #### *IDF(t) = N/n(t)*.
#### Os passos que você deve fazer é:
* #### Calcule *N*. Isso pode ser feito utilizando um método próprio dos RDDs.
* #### Crie um RDD contendo os tokens únicos de cada documento. Para cada documento, você deve incluir um token apenas uma vez, *mesmo que ele apareça mais de uma vez*.
* #### Para cada um dos tokens únicos, conte quantas vezes ele aparece no documento e então compute o IDF para o token: *N/n(t)*
#### Use seu `idfs` para calcular o IDF da sua base dataToToken.
#### Quantos tokens únicos temos?

In [ ]:
# EXERCICIO
def idfs(corpus):
    """ Compute IDF
    Args:
        corpus (RDD): input corpus
    Returns:
        RDD: a RDD of (token, IDF value)
    """
    N = corpus.count()
    uniqueTokens = corpus.<COMPLETAR>
    tokenCountPairTuple = uniqueTokens.<COMPLETAR>
    tokenSumPairTuple = tokenCountPairTuple.<COMPLETAR>
    return <COMPLETAR>

idfsData = idfs(dataToToken)
uniqueTokenCount = idfsData.count()

print 'There are %s unique tokens in the small datasets.' % uniqueTokenCount
print idfsData.takeOrdered(10, lambda s: s[1])

In [ ]:
assert uniqueTokenCount==14909, 'valor incorreto!'
print 'ok'
tokenSmallestIdf = idfsData.takeOrdered(1, lambda s: s[1])[0]
assert tokenSmallestIdf[0]==u'film', 'valor incorreto'
print 'ok'
assert abs(tokenSmallestIdf[1]-7.56028) < 0.00001, 'valor incorreto'
print 'ok'

### **(2d) Tokens com os menores IDF**
#### Imprima os 11 tokens com menor IDF.

In [ ]:
smallIDFTokens = idfsData.takeOrdered(11, lambda s: s[1])
print smallIDFTokens

### **(2e) IDF Histograma**
#### Plote um histograma dos valores de IDF.  Use a quantidade apropriada de buckets para o gráfico.

In [ ]:
import matplotlib.pyplot as plt

idf_values = idfsData.map(lambda s: s[1]).collect()
fig = plt.figure(figsize=(8,3))
plt.hist(idf_values, 50, log=True)
pass

### **(2f) Implemente a função TF-IDF**

#### Use sua função `tf` para implementar uma função `tfidf(tokens,idfs)` que recebe uma lista de tokens de um documento e um dicionário Python de pesos IDF e retorna um dicionário Python mapeando cada token de `tokens` para seu peso total TF-IDF.

#### Os passos de sua função devem ser:
* #### Calcule o TF para `tokens`
* #### Crie um dicionário Python onde cada token mapeia para o tf multiplicado pelo IDF desse token

#### Use a função `tfidf` para computar os pesos do primeiro registro da RDD  dataToToken. Primeiro precisamos converter os IDFs dessa base para um dicionário Python. Para isso utilize a ação [`collectAsMap()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collectAsMap).

In [ ]:
# EXERCICIO
def tfidf(tokens, idfs):
    """ Compute TF-IDF
    Args:
        tokens (list of str): input list of tokens from tokenize
        idfs (dictionary): record to IDF value
    Returns:
        dictionary: a dictionary of records to TF-IDF values
    """
    tfs = <COMPLETAR>
    tfIdfDict = <COMPLETAR>
    return tfIdfDict

firstDoc = dataToToken.take(1)[0][1]
idfsWeights = idfsData.collectAsMap()
rec_weights = tfidf(firstDoc, idfsWeights)

print 'O primeiro documento tem tokens e pesos:\n%s' % rec_weights

In [ ]:
# TEST Implement a TF-IDF function (2f)
assert rec_weights=={u'independent': 284.26666666666665, u'seeking': 177.66666666666666, u'quiet': 59.222222222222214, u'introspective': 473.7777777777777, u'entertaining': 15.44927536231884, u'worth': 18.70175438596491}, 'valores incorretos'
print "ok"

### **Parte 3: Similaridade do Cosseno**

#### Nosso dicionário de TF-IDF pode ser visto como uma representação do texto em forma de um vetor esparso. Cada elemento do vetor pode ser nulo ou conter um valor numérico representando a importância daquele token para o documento.

#### Uma métrica utilizada para esse tipo de representaçao é a **[similaridade do cosseno][cosine]** que calcula o cosseno do ângulo entre dois vetores para indicar quão próximose eles estão entre si, independente da escala:

#### $$ a \cdot b = \| a \| \| b \| \cos \theta $$
#### $ a \cdot b = \sum a_i b_i $ é o produto interno de dois vetores, e $ \|a\| = \sqrt{ \sum a_i^2 } $ é a norma de $ a $.

#### Então temos que a similaridade é:
#### $$ similaridade = \cos \theta = \frac{a \cdot b}{\|a\| \|b\|} = \frac{\sum a_i b_i}{\sqrt{\sum a_i^2} \sqrt{\sum b_i^2}} $$

#### Essa métrica indica que, se o ângulo entre dois documentos é pequeno, eles compartilham muitos tokens em comum, pois estão apontando mais ou menos para a mesma direção. Para esses casos, o cosseno do ângulo será alto. 

[cosine]: https://en.wikipedia.org/wiki/Cosine_similarity

### **(3a) Implemente os componentes de uma função `cosineSimilarity`**

#### Use as funções `tokenize` e `tfidf`, e os pesos IDF da Parte 2 para extrair os tokens e atribuir pesos a eles.

#### Os passos a serem realizados devem ser:
* #### Defina uma função `dotprod` que recebe dois dicionários Python como parâmetro e calcula o produto interno deles, onde o produto interno é definido como a soma do produto dos valores para os tokens que aparecem em *ambos* os dicionários
* #### Defina uma função `norm` que retorna a raíz quadrada do produto interno do dicionário com ele mesmo
* #### Defina uma função `cossim` que retorna o produto interno entre dois dicionários dividido pelo produto das normas desses dicionários

In [ ]:
# EXERCICIO
import math

def dotprod(a, b):
    """ Compute dot product
    Args:
        a (dictionary): first dictionary of record to value
        b (dictionary): second dictionary of record to value
    Returns:
        dotProd: result of the dot product with the two input dictionaries
    """
    return <COMPLETAR>

def norm(a):
    """ Compute square root of the dot product
    Args:
        a (dictionary): a dictionary of record to value
    Returns:
        norm: a dictionary of tokens to its TF values
    """
    return <COMPLETAR>

def cossim(a, b):
    """ Compute cosine similarity
    Args:
        a (dictionary): first dictionary of record to value
        b (dictionary): second dictionary of record to value
    Returns:
        cossim: dot product of two dictionaries divided by the norm of the first dictionary and
                then by the norm of the second dictionary
    """
    return <COMPLETAR>

testVec1 = {'foo': 2, 'bar': 3, 'baz': 5 }
testVec2 = {'foo': 1, 'bar': 0, 'baz': 20 }
dp = dotprod(testVec1, testVec2)
nm = norm(testVec1)
print dp, nm

In [ ]:
# TEST Implement the components of a cosineSimilarity function (3a)
assert dp==102, 'valor incorreto'
print 'ok'
assert abs(nm - 6.16441400297) < 0.0000001, 'valor incorreto'
print 'ok'

### **(3b) Função `cosineSimilarity` **

#### Implemente a função `cosineSimilarity(string1, string2, idfsDictionary)`  que recebe duas strings e um dicionário de pesos IDF e retorna a similaridade do cosseno para essas strings.

#### Os passos a serem realizados:
* #### Aplique sua função `tfidf` para a primeira e segunda strings tokenizadas (pela função `tokenize`) e usando o dicionário de pesos IDF
* #### Aplique a função `cossim` retornando seu valor

In [ ]:
# EXERCICIO
def cosineSimilarity(string1, string2, idfsDictionary):
    """ Compute cosine similarity between two strings
    Args:
        string1 (str): first string
        string2 (str): second string
        idfsDictionary (dictionary): a dictionary of IDF values
    Returns:
        cossim: cosine similarity value
    """
    w1 = <COMPLETAR>
    w2 = <COMPLETAR>
    return cossim(w1, w2)

cossimreview = cosineSimilarity('this movie is good',
                               'this movie is bad',
                               idfsWeights)

print cossimreview

In [ ]:
# TEST Implement a cosineSimilarity function (3b)
assert abs(cossimreview - 0.04446) < 0.0001, 'valor incorreto'
print 'ok'

### **(3c) Similaridade entre documentos**

#### Agora podemos finalmente calcular a similaridade entre os documentos de nossa base.

#### Para gerar a matriz de distância entre cada documento vamos fazer os seguintes passos:

* #### Crie uma RDD que é a combinação de cada registro da base dataRDD com ela mesma, com isso teremos uma RDD no formato `[ ( (id1, string1), (id2, string2) ),  ( (id1, string1), (id3, string3) ), ... ]`
* #### Crie uma função que calcule a similaridade do cosseno para um dado registro dessa RDD combinada e utilizando uma variável broadcast para os pesos idfs
* #### Crie uma variável broadcast com os pesos IDFs calculados
* #### Aplique essa função na RDD

#### NOTA: alguns bugs irão ocorrer nas funções tf e cosineSimilarity, vocês devem ler a mensagem de erro e corrigir tais códigos conforme necessário.

In [ ]:
# EXERCICIO
samplesRDD = dataRDD.sample(False, 0.1, 42).cache()

crossRDD = (samplesRDD
              .<COMPLETAR>
              .cache())

def computeSimilarity(record):
    """ Compute similarity on a combination record
    Args:
        record: a pair, (rec1, rec2)
    Returns:
        pair: a pair, (rec1_id, rec2_id, cosine similarity value)
    """    
    <COMPLETAR>
    return (rec1_id, rec2_id, cs)

idfsWeightsBroadcast = sc.broadcast(idfsWeights)
similarities = (crossRDD
                .map(computeSimilarity)
                .cache())

print similarities.take(10)

In [ ]:
assert similarities.take(1)[0][2]==1.0, 'valores incorretos'
print 'ok'

### **Part 4: k-NN escalável**

#### Na parte anterior, o cálculo de similaridade entre dois documentos está limitado a uma complexidade quadrática, o que não é prática em bases de dados de tamanho moderado. Vamos implementar uma forma escalável de calcular as distâncias entre pares de documentos.

### Índices invertidos

#### Notem que boa parte dos documentos tem similaridade igual a zero, ou seja, não possuem tokens em comum. Para resolver esse problema podemos utilizar a estrutura de dados chamada [**índice invertido**](https://en.wikipedia.org/wiki/Inverted_index) que mapeia cada token na base de dados a lista de documentos que os contém. Então ao invés de compararmos todos os pares de registros para verificar se eles contém tokens em comum, nós iremos nos restringir apenas aos pares de documentos que tem algo em comum.

### **(4a) Tokenize a base de dados completa**
#### Tokenize a base dataRDD para pré-processar os tokens.

In [ ]:
# EXERCICIO
tokenRDD = dataRDD.<COMPLETAR>
print 'A base de dados contém %s itens' % (tokenRDD.count())

In [ ]:
assert tokenRDD.count() == 8528, 'valores incorretos'
print 'ok'

### **(4b) Calcule os TF-IDFs para a base de dados **
#### Vamos utilizar as funções já criadas e a variável de broadcast definida anteriormente (`idfsWeightsBroadcast`)
#### Os passos a serem feitos:
* #### Crie um novo `dataTFIDFRDD` primeiro mapeando os tokens de tokenRDD para os pesos tf
* #### Em seguida, aplique os pesos idf para cada token, gerando um dicionário (token, tfidf).

In [ ]:
# EXERCICIO
dataTFIDFRDD = (tokenRDD
                .<COMPLETAR>
                .<COMPLETAR>
               )

print 'There are %s weights .' % (dataTFIDFRDD.count())

In [ ]:
assert dataTFIDFRDD.count()==8528, 'valores incorretos'
print 'ok'

### **(4c) Pré-calculando as normas para os pesos da base**

* #### Crie uma RDD de tuplas em que temos (id, norm(tokens))
* #### Converta essa RDD em uma variável de broadcast como um dicionário de normas

In [ ]:
# EXERCICIO
dataNorms = dataTFIDFRDD.map(lambda rec: (rec[0],norm(rec[1])))
dataNormsBroadcast = sc.broadcast(dataNorms.collectAsMap())

### **(4d) Crie um índice invertido para a base completa**

* #### Crie uma função `invert` que recebe um registro da base e gera uma lista de tuplas (token,(id, tfidf)) para cada token do registro. Lembre-se que o valor do RDD de tuplas é um dicionário com chaves iguais aos tokens e valores igual ao peso TF-IDF.

* #### Use essa função para converter nossa RDD dataTFIDFRDD para uma RDD de índices invertidos.

In [ ]:
# EXERCICIO
def invert(record):
    """ Invert (ID, tokens) to a list of (token, ID)
    Args:
        record: a pair, (ID, token vector)
    Returns:
        pairs: a list of pairs of token to ID
    """
    <COMPLETAR>
    return (pairs)

dataInvPairsRDD = (dataTFIDFRDD
                    .<COMPLETAR>
                    .cache())

print 'There are %s inverted pairs.' % (dataInvPairsRDD.count())

In [ ]:
assert dataInvPairsRDD.count() == 79658, 'valores incorretos'
print 'ok'

### **(4e) Identifique tokens em comum nos registros da base de dados**

* #### Usando o índice invertido, agrupe a base pela chave (token) e aplique a função `genList` que deve gerar uma lista de tuplas (token, ((doc1,tfidf),(doc2,tfidf)) de todos os pares de documentos com essa token em comum exceto nos casos `doc1==doc2`.
* #### Em seguida, gere tuplas do tipo ( (doc1, doc2), tfidf1*tfidf2/(norma1*norma2) ) a reduza para realizar a somatória desses valores sob a mesma chave.

#### Dessa forma teremos os registros de pares de documentos que possuem similaridade não nula com sua similaridade calculada.

In [ ]:
# EXERCICIO
from itertools import product

def genList(record):
    """ generate a list of tuple of documents and a token
    Args:
        record: a pair, (token, [list of documents with tfidf])
    Returns:
        list: [( tuple, token)]
    """
    <COMPLETAR>
    return <COMPLETAR>

def sumTFIDF(record):
    """ Multiply the tfidf of both documents, to a given term and divide by the norm.
    Args:
        record: a pair, (((doc1, tfidf), (doc2, tfidf)), token)
    Returns:
        pair: ((ID, URL), tfidf)
    """   
    <COMPLETAR>
    return (key, value)

commonTokens = (dataInvPairsRDD
                .<COMPLETAR>
                .<COMPLETAR>
                .<COMPLETAR>
                .<COMPLETAR>
                .cache()
                )

#print 'Found %d common tokens' % commonTokens.count()
print commonTokens.take(2)
print commonTokens.count()

In [ ]:
assert commonTokens.count()==5620224, 'valores incorretos'
print 'ok'

### **(4f) k-NN**

#### Vamos gerar agora a lista dos *k* documentos mais similares de cada documento.

* #### Gere uma RDD partindo da `commonTokens` de tal forma a ter ( id1, (id2, sim) )
* #### Agrupe pela chave e transforme a RDD para ( id1, [ (id,sim) ] ) onde a lista deve ter k elementos

In [ ]:
# EXERCICIO
def genklist(rec,k):
    """ Generate the list of the k most similar documents to the key
    Args:
        record: a pair, (doc, [(doc,sim)])
        k: number of most similar elements
    Returns:
        pair: (doc, [(doc,sim)])
    """
    <COMPLETAR>
    return (key, docs[:k])
    
def knn(simRDD, k):
    """ Generate the knn RDD for a given RDD.
    Args:
        simRDD: RDD of ( (doc1,doc2), sim)
        k: number of most similar elements
    Returns:
        RDD: RDD of ( doc1, [docs, sims])
    """

    ksimRDD = (simRDD
               .<COMPLETAR>
               .<COMPLETAR>
               .<COMPLETAR>
              )
    return ksimRDD

ksimReviewsRDD = knn(commonTokens, 3)
ksimReviewsRDD.take(3)

In [ ]:
print dataRDD.filter(lambda x: x[0] in [8198,12615,30826,46764]).collect()